# Visualize memory

In [1]:
import sys
import os

# Get the absolute path of the parent directory.
parent_dir = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))

# Add the parent directory to the system path to be able to import modules from 'lib.'
sys.path.append(parent_dir)

In [2]:
%%capture

import gensim.downloader as api
from IPython.display import Markdown as md

from lib.memory import DSDM
from lib.utils import cleanup, configs, inference, learning, preprocess, utils

import numpy as np
import pandas as pd
import pickle
import plotly.express as px
from sklearn.manifold import TSNE
import subprocess

In [43]:
display(md('#### Available memories:'))
print(f"{subprocess.getoutput('ls ./memories/method2 | nl')}")

#### Available memories:

     1	memory_2023-09-09 22-24-26-309400.pkl
     2	memory_2023-09-09 23-06-41-117607.pkl
     3	memory_2023-09-10 09-21-47-014390.pkl
     4	memory_2023-09-10 13-01-52-622408.pkl
     5	memory_2023-09-10 13-09-22-030328.pkl
     6	memory_2023-09-10 17-02-12-946618.pkl
     7	memory_2023-09-10 19-02-18-496164.pkl
     8	memory_2023-09-10 20-04-56-643706.pkl


In [87]:
#number = int(input())
#command = "ls ./{0}/ | sed '{1}!d'"

# Load cleanup and memory
# cleanup = pickle.load(open(f"cleanups/{subprocess.getoutput(command.format('cleanups', number))}",'rb'))
# memory = pickle.load(open(f"memories/{subprocess.getoutput(command.format('memories', number))}",'rb'))

cleanup = pickle.load(open(f"cleanups/method2/cleanup_2023-09-10 13-09-22-030328.pkl",'rb'))
memory = pickle.load(open(f"memories/method2/memory_2023-09-10 13-09-22-030328.pkl",'rb'))

### Memory stats

In [88]:
len(memory.wiki_articles)

5010

In [89]:
len(memory.addresses)

136896

In [90]:
memory.ema_time_period

100000

In [91]:
memory.n_updates

234448

In [92]:
memory.n_expansions

136896

In [85]:
memory.n_deletions

0

In [86]:
len(memory.wiki_articles)

12

In [23]:
print(f"Memory size: {len(memory.addresses)}")

Memory size: 105240


In [ ]:
%%capture

# Load pre-trained word embeddings (Word2Vec in this example)
word_vectors = api.load("word2vec-google-news-300")

In [ ]:
%%capture
concepts = [""]#, "universities"]#, 'days', 'day', 'record', 'stage']

address_embeddings = []
address_concepts = []
addresses = []
bins = []
chunk_scores = []

for idx, address in enumerate(memory.addresses[0:1000]):
    tokens = inference.get_most_similar_HVs(inference.get_similarities_to_atomic_set(address, cleanup))
    embeddings = [word_vectors[word] for word in tokens if word in word_vectors]
    if len(set(concepts).intersection(set(tokens))) > 0:
    #if embeddings:
        addresses.append(idx)
        bins.append(memory.scores[idx, 1].item())
        chunk_scores.append(memory.scores[idx, 0].item())
        address_concepts.append(" ".join(tokens))
        address_embeddings.append(sum(embeddings) / len(embeddings))

In [ ]:
address_concepts

In [ ]:
reduced_embeddings = TSNE(n_components=2, random_state=42, perplexity=2).fit_transform(np.array(address_embeddings))

df = pd.DataFrame(reduced_embeddings, columns=["Dimension 1", "Dimension 2"])
df["Address"] = addresses
df["Chunk"] = address_concepts
df['Bin'] = bins
df['Chunk-score'] = chunk_scores

In [ ]:
fig = px.scatter(
    df, x="Dimension 1", y="Dimension 2",
    text="Chunk", hover_data=["Address", "Bin", "Chunk-score"],
    title="Memory concepts"
)
fig.show()

In [ ]:
memory.scores[294]

In [ ]:
_df = pd.DataFrame()

In [42]:
addresses = np.random.randint(0, len(memory.addresses), size=70)
# addresses = np.argwhere((memory.chunk_scores > 0.97).cpu().detach().numpy().flatten()).flatten()
#addresses = np.argwhere((memory.scores[:, 1] > 4).cpu().detach().numpy().flatten()).flatten()
# addresses = [3, 4]

for address in addresses:
    display(md(f"### Address {address}"))
    address_sims_df = inference.get_similarities_to_atomic_set(
            memory.addresses[address],
            cleanup,
    )
    display(address_sims_df)

### Address 2575

,token,similarity
0,as,0.70
1,a,0.70
2,membership,0.11
3,catalonia,0.10
4,member,0.08
5,external,0.08
6,penelope,0.08
7,beach,0.08
8,christopher,0.08
9,committee,0.08


### Address 1593

,token,similarity
0,ridge,0.70
1,east,0.70
2,2011,0.10
3,leaders,0.10
4,book,0.09
5,turnover,0.09
6,sunday,0.09
7,nieuw,0.09
8,vicissitudes,0.09
9,city,0.08


### Address 2668

...

,token,similarity
0,debut,0.70
1,on,0.70
2,yeast,0.13
3,wrote,0.10
4,involved,0.09
5,turnover,0.09
6,belief,0.08
7,sea,0.08
8,white,0.08
9,to,0.07


### Address 2121

,token,similarity
0,the,0.60
1,ball,0.58
2,spilled,0.58
3,illuminated,0.10
4,netting,0.09
5,convex,0.08
6,sisters,0.08
7,runner,0.08
8,sliced,0.08
9,additional,0.08


### Address 1864

,token,similarity
0,the,0.59
1,route,0.58
2,to,0.56
3,bruno,0.10
4,measures,0.09
5,town,0.08
6,temperance,0.08
7,additional,0.08
8,12,0.08
9,went,0.08
